<h1> NEAT Implementation Practice </h1>
A first-pass attempt at implementing NEAT by translating SethBling's neatevolve.lua script into Python.

# Some Notes
Here are some things about which knobs you can turn in each cell:
1. Imports. Don't touch anything there unless you wanna try something out at your own risk.
2. Environment Declarations. Change the envName string if you want to test out other environments, but just note that you may need to change some other parameters in the evaluation cell as well if you do so.
3. Hyperparameters. Change as you please, but I would recommend sticking to the values from the NEAT paper and/or neatevolve.lua.
4. Some functions. I wouldn't touch this.
5. Genome Class. I wouldn't touch this, but please feel free to look at the mess I've made.
6. Crossover Function. Wouldn't recommend touching this.
7. Delta Function. Wouldn't touch this one either.
8. Evaluation. As mentioned in 2), if you change the environment, then you made need to change the evaluateAction method of the genome or some of the target fitness requirements for episode termination.

In [1]:
import gym
import numpy as np # might not even need this ?
import random # pseudo-random|
import math
import time # for timing how long it runs for 

In [17]:
# first define the environment and get the both the action and observation spaces 
envName = 'BipedalWalker-v3' # just need to change this if you wanna try it in environments
env = gym.make(envName)

# print out the observation and action spaces
print(env.action_space.shape[0]) # number of valid actions we can take 
print(env.observation_space.shape[0]) # number of "sensors" of the environment that we have (number of inputs)

# NOTE: Need to keep track of the valid values these can take 
# NOTE: Also need to have inputs + 1, with the 1 being the bias (stc?)
# NOTE: changes between Box() and Discrete() depending on environment
outputs = env.action_space.shape[0] 
inputs = env.observation_space.shape[0] + 1


4
24


In [7]:
# hyperparameters, according to section 4.1 Parameter Settings (changed some parameters for myself)
deltaThreshold = 1.0

c1 = c2 = 2.0
c3 = 0.4
perturbVal = 0.05


numUntilStagnant = 15


genomeMutationChance = 0.25 # neatevolve.lua has this pretty low 
connectionUniformPerturb = 0.9
connectionRandomValue = 1 - connectionUniformPerturb
disableGeneChance = 0.75 
mutateNoCrossoverChance = 0.25
interspeciesMatingRate = 0.01 
smallPopulationNewNodeChance = 0.03
largePopulationNewNodeChance = 0.05 # using these right now 
smallPopulationNewConnectionChance = 0.05
largePopulationNewConnectionChance = 0.3 # using these right now 

In [8]:
# activation functions 
def sigmoid(x):
    return 1/(1+np.exp(-x))

def steepsigmoid(x):
    return 1/(1+np.exp(-4.9*x))

In [19]:
# genome class 
class Genome:
    # initialize class variables 
    globalInnovationNumber = 1
    def __init__(self, inputs=1, outputs=1):
        self.globalInnovation = 1
        self.nodeGenes = {}
        self.connectGenes = {}
        self.fitness = 0
        self.sharedFitness = 0
        self.species = 0
        self.inputCount = inputs
        self.outputCount = outputs
        self.mutationSuccess = False 
        
    def setGenes(self, nodeGenes, connectGenes):
        self.nodeGenes = nodeGenes.copy()
        self.connectGenes = connectGenes.copy()
        
    @staticmethod
    def copyGenome(genome):
        g = Genome(genome.inputCount, genome.outputCount)
        g.setGenes(genome.nodeGenes, genome.connectGenes)
        return g
    
    @staticmethod
    def resetGlobalInnovationNumber():
        Genome.globalInnovationNumber = 1
        # print('Genome global innovation number reset back to: {}'.format(Genome.globalInnovationNumber))
    
    def clearGenome(self):
        self.globalInnovation = 1
        self.nodeGenes.clear()
        self.connectGenes.clear()
        
    def clearNodeValues(self):
        for node in self.nodeGenes:
            self.nodeGenes[node][1] = 0.0
            
    def initEmptyGenome(self):
        self.clearGenome()
        for i in range(self.inputCount):
            self.insertNode('input')
        for i in range(self.outputCount):
            self.insertNode('output')
            
    def resetFitness(self):
        self.fitness = 0
        self.sharedFitness = 0
        
    def initRandomGenome(self):
        self.initEmptyGenome()
        
        # brute force connect each new input node to each output node 
        for node in self.nodeGenes:
            if self.nodeGenes[node][0] == 'input':
                for otherNode in self.nodeGenes:
                    if self.nodeGenes[otherNode][0] == 'output':
                        self.insertConnection(outNode=node, inNode=otherNode, weight=random.uniform(-1, 1))
             
    def insertNode(self, nodeType='input', value = 0.0):
        self.nodeGenes[len(self.nodeGenes)+1]=[nodeType, value]
        
    def insertConnection(self, outNode, inNode, weight = 0.5, isExpressed = True):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes): # check if the nodes even exist 
            if ((outNode, inNode) not in self.connectGenes and (inNode, outNode) not in self.connectGenes and self.nodeGenes[inNode][0] != 'input' and outNode != inNode and self.nodeGenes[outNode][0] != 'output'): # check that the connection doesnt already exist
                # also dont allow recurrent connections (only feedforward)
                self.connectGenes[(outNode, inNode)] = [Genome.globalInnovationNumber, weight, isExpressed]
                Genome.globalInnovationNumber += 1
                self.mutationSuccess = True
            else:
                return
                #print('Connection already exists. Did not insert')
        else:
            return
            #print('Could not find node(s). Did not insert.')
            
    def checkConnection(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                return self.connectGenes[(outNode, inNode)][2]
            else:
                print('Could not find the connection')
        else:
            print('Could not find node(s). Did not disable.')
            
    def disableConnection(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                self.connectGenes[(outNode, inNode)][2] = False
            else:
                print('Could not find connection to disable. Did not disable.')
        else: 
            print('Could not find node(s). Did not disable.')
    
    def mutateAddConnection(self, outNode, inNode):
        self.insertConnection(outNode, inNode, random.uniform(-1, 1), True)
            
    def mutateAddRandomConnection(self):
        timesAttemptedToConnected = 0
        self.mutationSuccess = False
        while (not self.mutationSuccess):
            self.mutateAddConnection(random.choice(list(self.nodeGenes)),random.choice(list(self.nodeGenes)))
            timesAttemptedToConnected +=1
            if (timesAttemptedToConnected > 100):
                print('Attempted to connect too many times. probably already full')
                return
        self.mutationSuccess = False
        
    def mutateAddNode(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                self.insertNode('hidden', '0.0')
                # disable old connection'
                self.disableConnection(outNode, inNode)
                self.insertConnection(outNode, len(self.nodeGenes), 1.0, True)
                # get original weight 
                originalWeight = self.connectGenes[(outNode, inNode)][1]
                self.insertConnection(len(self.nodeGenes), inNode, originalWeight, True)
                self.mutationSuccess = True
                print('added hidden node')
            else:
                print('Error! No connection found.')
        else:
            print('Could not find node(s). Did not mutate.')
    
    def mutateAddRandomNode(self):
        if (not self.connectGenes): # need to debug this 
            print('connectGenes is empty. can\'t mutateAddRandomNode')
            return
        self.mutationSuccess = False
        while (not self.mutationSuccess):
            randomTuple = random.choice(list(self.connectGenes))
            self.mutateAddNode(randomTuple[0],randomTuple[1])
        self.mutationSuccess = False
    
    # this is so fucking slow
    def evaluateAction(self, observation):
        # TODO: IMPLEMENT THIS
        if not self.connectGenes:
            return env.action_space.sample()
        else:
            self.clearNodeValues()
            self.nodeGenes[1][1] = 1.0
            for i in range(self.inputCount-1):
                # print('Loading value: {} into node input: {}'.format(observation[i], i+2))
                self.nodeGenes[i+2][1] = observation[i]
            
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'hidden': # for every hidden node
                    x = []
                    w = []
                    for connection in self.connectGenes:
                        if connection[1] == node and self.connectGenes[connection][2] == True: # if inNode == node
                            x.append(self.nodeGenes[connection[0]][1])
                            w.append(self.connectGenes[connection][1])
                    self.nodeGenes[node][1] = steepsigmoid(np.dot(x, w))
                    # print('hidden node #{}\'s value: {}'.format(node, self.nodeGenes[node][1]))
            softmaxsum = 0
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'output':
                    x = []
                    w = []
                    for connection in self.connectGenes:
                        if connection[1] == node and self.connectGenes[connection][2] == True:
                            x.append(self.nodeGenes[connection[0]][1])
                            w.append(self.connectGenes[connection][1])
                    self.nodeGenes[node][1] = np.exp(np.dot(x, w))
                    softmaxsum += np.exp(np.dot(x, w))
                    # print('output node #{}\'s unweighted value: {}'.format(node, self.nodeGenes[node][1]))
            output = []
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'output':
                    output.append(self.nodeGenes[node][1] / softmaxsum)
            # print(output)
            # print('Genome takes action {}'.format(output.index(max(output))))
            return output.index(max(output))
        
    def evaluateActionBox(self, observation):
        # TODO: IMPLEMENT THIS
        if not self.connectGenes:
            return env.action_space.sample()
        else:
            self.clearNodeValues()
            self.nodeGenes[1][1] = 1.0
            for i in range(self.inputCount-1):
                # print('Loading value: {} into node input: {}'.format(observation[i], i+2))
                self.nodeGenes[i+2][1] = observation[i]
            
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'hidden': # for every hidden node
                    x = []
                    w = []
                    for connection in self.connectGenes:
                        if connection[1] == node and self.connectGenes[connection][2] == True: # if inNode == node
                            x.append(self.nodeGenes[connection[0]][1])
                            w.append(self.connectGenes[connection][1])
                    self.nodeGenes[node][1] = steepsigmoid(np.dot(x, w))
                    # print('hidden node #{}\'s value: {}'.format(node, self.nodeGenes[node][1]))
            softmaxsum = 0
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'output':
                    x = []
                    w = []
                    for connection in self.connectGenes:
                        if connection[1] == node and self.connectGenes[connection][2] == True:
                            x.append(self.nodeGenes[connection[0]][1])
                            w.append(self.connectGenes[connection][1])
                    self.nodeGenes[node][1] = np.exp(np.dot(x, w))
                    softmaxsum += np.exp(np.dot(x, w))
                    # print('output node #{}\'s unweighted value: {}'.format(node, self.nodeGenes[node][1]))
            output = []
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'output':
                    output.append(self.nodeGenes[node][1] / softmaxsum)
            # print(output)
            # print('Genome takes action {}'.format(output.index(max(output))))
            return output
    
    def evaluateSharedFitness(self, speciesCount):
        self.sharedFitness = self.fitness / speciesCount
    
    def printNodeGenes(self):
        print(self.nodeGenes)
        
    def printNodeGeneCount(self):
        print(len(self.nodeGenes))
        
    def printConnectGenes(self):
        print(self.connectGenes)
    
    def printConnectGeneCount(self):
        print(len(self.connectGenes))
    
    def printActiveConnectGenes(self):
        for connection in self.connectGenes:
            if (self.connectGenes[connection][2]):
                print(connection, self.connectGenes[connection][1])
        
    def printGlobalInnovation(self):
        print(self.globalInnovation)

In [10]:
def crossover(genome1, genome2):
    if (genome2.fitness > genome1.fitness):
        temp = genome1
        genome1 = genome2
        genome2 = temp
    
    g = Genome(genome1.inputCount, genome1.outputCount)
    g.initEmptyGenome()
    g1GenomeList = {}
    for connection in genome1.connectGenes:
        #print(genome1.connectGenes[connection])
        g1GenomeList[genome1.connectGenes[connection][0]]=[connection,genome1.connectGenes[connection][1],genome1.connectGenes[connection][2]]
    g2GenomeList = {}
    for connection in genome2.connectGenes:
        g2GenomeList[genome2.connectGenes[connection][0]]=[connection,genome2.connectGenes[connection][1],genome2.connectGenes[connection][2]] 
        
    # when crossing over, the genes in both genomes with the same innovation numbers are lined up
    # in composing the offspring, genes are randomly chosen from either parent at matching genes,
    # whereas all excess or disjoint genes are always included from the more fit parent 
    for i in range(max(len(g1GenomeList), len(g2GenomeList))):
        if (i+1 in g1GenomeList and i+1 in g2GenomeList):
            if (random.random() <= 50): # inherit from genome1
                # print('inherited from parent 1')
                g.connectGenes[g1GenomeList[i+1][0]] = [i+1, g1GenomeList[i+1][1], g1GenomeList[i+1][2]]
            else: # inherit from genome2 
                # print('inherited from parent 2')
                g.connectGenes[g2GenomeList[i+1][0]] = [i+1, g2GenomeList[i+1][1], g2GenomeList[i+1][2]]
            if (g1GenomeList[i+1][2] == False or g1GenomeList[i+1][2] == False):
                if (random.random() <= disableGeneChance):
                    # print('inherited disabled gene')
                    g.connectGenes[g1GenomeList[i+1][0]][2] = False
                else:
                    # print('re-enabled inherited gene')
                    g.connectGenes[g1GenomeList[i+1][0]][2] = True
        if (i+1 in g1GenomeList and i+1 not in g2GenomeList):
            g.connectGenes[g1GenomeList[i+1][0]] = [i+1, g1GenomeList[i+1][1], g1GenomeList[i+1][2]]
    
    
    # insert the node genes 
#     for n in range(g.inputCount)
    for c in g.connectGenes:
        if c[0] in g1.nodeGenes:
            g.nodeGenes[c[0]] = g1.nodeGenes[c[0]]
        elif c[0] in g2.nodeGenes:
            g.nodeGenes[c[0]] = g2.nodeGenes[c[0]]
            
        if c[1] in g1.nodeGenes:
            g.nodeGenes[c[1]] = g1.nodeGenes[c[1]]
        elif c[0] in g2.nodeGenes:
            g.nodeGenes[c[1]] = g2.nodeGenes[c[1]]
            
    g.clearNodeValues()
    return g
    

In [11]:
# TODO: Verify this works properly
def delta(genome1, genome2, c1=0.1, c2=0.1, c3=0.1):
    if (not genome1.connectGenes or not genome2.connectGenes):
        return 0
    connectionMismatch = []
    for connection1 in genome1.connectGenes:
        if (connection1 not in genome2.connectGenes):
            connectionMismatch.append(connection1)
    for connection2 in genome2.connectGenes:
        if (connection2 not in genome1.connectGenes):
            connectionMismatch.append(connection2)
    diffsum = 0
    num = 0
    for connection1 in genome1.connectGenes:
        if (connection1 not in connectionMismatch):
            diffsum += abs(genome1.connectGenes[connection1][1] -
                           genome2.connectGenes[connection1][1])
            num += 1
    averageWeightDifference = diffsum / num
    #print(averageWeightDifference)
    N = 0
    if len(genome1.connectGenes) >= len(genome2.connectGenes):
        N = len(genome1.connectGenes)
    else:
        N = len(genome2.connectGenes)
    # print(((len(connectionMismatch)*(c1 + c2))/N) + (c3*averageWeightDifference))
    return ((len(connectionMismatch) *
             (c1 + c2)) / N) + (c3 * averageWeightDifference)

> connection weights mutate as in any NE system, with each connection either perturbed or not at each generation 

In [ ]:
max_generations = 50
population_limit = 300
championThresholdCount = 5


species = {}
pop = {}

# NOTE: input/output changes depending on environment !
inputs = env.observation_space.shape[0] + 1
outputs = env.action_space.shape[0] 


targetFitness = 300 # this is for Bipedal-v3
maxTestTrials = 100
targetAverageFitness = 300 # this is for Bipedal-v3
targetPosition = 0.5 # MountainCar-v0

solved = False

globalLoopCounter = 0
bestGenome = None
highestFitness = float("-inf")
start_time = time.time()
for generation in range(max_generations):
    print('Beginning Generation #{}'.format(generation+1))
    if (generation == 0): # start out with a uniform population of networks with zero hidden networks
        for genome in range(population_limit):
            g = Genome(inputs, outputs)
            Genome.resetGlobalInnovationNumber()
            g.initRandomGenome()
            pop[genome+1] = g
    
    # in each generation, genomes are sequentially placed into species 
    
    fitness = []
    
    for genome in pop:
        print('Beginning evaluation on genome #{}'.format(genome))
        if (not species):
            print('creating new species:{}'.format(len(species)+1))
            species[len(species)+1] = {'summedSharedFitness': 0.0, 'maxFitness':0.0, 'maxAdjustedFitness': 0.0, 'stagnantCounter' : 0, 'isStagnant':False,'list':[pop[genome]], 'bestGenome': pop[genome]}
            pop[genome].species = len(species)
            print(len(species))
        else:
            for og in species:
                if (delta(pop[genome], species[og]['list'][0], c1, c2, c3) < deltaThreshold): # just compare with the first species
                    print('inserting into species#{}'.format(og))
                    species[og]['list'].append(pop[genome])
                    pop[genome].species = og
                    break
                else:
                    if (og == len(species)):
                        print('creating new species:{}'.format(len(species)+1))
                        species[len(species)+1] ={'summedSharedFitness': 0.0, 'maxFitness':0.0, 'maxAdjustedFitness': 0.0, 'stagnantCounter': 0, 'isStagnant':False,'list':[pop[genome]], 'bestGenome': pop[genome]}
                        pop[genome].species = len(species)
                        print(len(species))
                        break
                        
        observation = env.reset()
        pop[genome].resetFitness()
        # evalute the genome's performance # SDHJKLSDHLSJKDHLSDHJKLHDSKLDHJ
        t = 0
        globalLoopCounter += 1
        while(True):
            t+=1
            env.render()
            action = pop[genome].evaluateActionBox(observation)
            observation, reward, done, info = env.step(action)
            pop[genome].fitness += reward
            if done:
                print("Episode finished after {} timesteps".format(t+1))
                print("Genome fitness: {}".format(pop[genome].fitness))
                fitness.append(pop[genome].fitness)
                if pop[genome].fitness > species[pop[genome].species]['maxFitness']:
                    species[pop[genome].species]['maxFitness'] = pop[genome].fitness
                    species[pop[genome].species]['stagnantCounter'] = 0
                    species[pop[genome].species]['isStagnant'] = False
                if pop[genome].fitness > highestFitness:
                    bestGenome = pop[genome]
                    highestFitness = pop[genome].fitness
                if pop[genome].fitness >= targetFitness: # changes depending on environment !
                    print('Found an optimal genome. testing it...')
                    testBenchTotalFitness = 0
                    for i in range(maxTestTrials):
                        m = 0
                        pop[genome].resetFitness()
                        observation = env.reset()
                        while(True):
                            m+=1
                            env.render()
                            action = pop[geome].evaluateActionBox(observation)
                            observation, reward, done, info = env.step(action)
                            bestGenome.fitness += reward
                            if done:
                                testBenchTotalFitness+=pop[genome].fitness
                                print("Genome fitness: {}".format(pop[genome].fitness))
                                break
                    print('average fitness across 100 trials:{}'.format(testBenchTotalFitness/maxTestTrials))
                    if (testBenchTotalFitness/maxTestTrials >= targetAverageFitness):
                        print('found solution genome')
                        solved = True
                break
                
    if (solved):
        break
    
    
    
    print('Number of species at generation {}: {}'.format(generation+1, len(species)))
    # print('highest achieved score in this generation {}'.format(max(fitness)))
     
    # print(species[1])
    adjustedFitness = []
    totalAdjustedFitness = 0.0
    for genome in pop:
        pop[genome].evaluateSharedFitness(len(species[pop[genome].species]))
        adjustedFitness.append(pop[genome].sharedFitness)
        species[pop[genome].species]['summedSharedFitness'] += pop[genome].sharedFitness
        if pop[genome].sharedFitness > species[pop[genome].species]['maxAdjustedFitness']:
            species[pop[genome].species]['maxAdjustedFitness'] = pop[genome].sharedFitness
            species[pop[genome].species]['bestGenome'] = pop[genome]
            
    for s in species:
        totalAdjustedFitness += species[s]['summedSharedFitness']
        print('Maximum adjusted fitness in species {}: {}'.format(s, species[s]['maxAdjustedFitness']))
        print('Summed adjusted fitness in species {}: {}'.format(s, species[s]['summedSharedFitness']))
    
    # print('Total Adjusted Fitness across all species: {}'.format(totalAdjustedFitness))
    
    newPop = {}
    
    # the champion of each species with more than 5 networks will be copied into the next generation unchanged
    i = 0
    for s in species:
        if len(species[s]['list']) > championThresholdCount:
            newPop[i+1] = Genome.copyGenome(species[s]['bestGenome'])
            i += 1
    
    print('New Pop count after copying champions only: {}'.format(len(newPop)))
    print('i\'s value after inserting champions: {}'.format(i))
            
    
    # getting the proportion of offspring they are allowed
    totalBreedCount = 0
    for s in species:
        if species[s]['isStagnant']:
            print('species #{} is stagnant'.format(s))
            species[s]['breedAmountAllowed'] = 0
        else:
            species[s]['breedAmountAllowed'] = np.floor((species[s]['summedSharedFitness']/totalAdjustedFitness)*(population_limit-i)).astype(int)
            totalBreedCount += species[s]['breedAmountAllowed']

    
    leftovers = population_limit - i - totalBreedCount
    print('leftovers:{}'.format(leftovers))
        
    # get rid of the "lowest performing members" of each species (TODO)
    for s in species:
        if (len(species[s]['list']) > 1): # only get rid of the bottom 75 percent of each species
            amountToCull = np.floor((.75)*len(species[s]['list'])).astype(int)
            print('Eliminating {} genomes from species#{}'.format(amountToCull, s))
            # sort from ascending to descending 
            species[s]['list'].sort(reverse = False, key=lambda genome: genome.fitness)
            for q in range(amountToCull):
                species[s]['list'].pop()
    
    
    # now for each species breed !
    for s in species:
        print('species {} is allowed to have {} offspring'.format(s, species[s]['breedAmountAllowed']))
        for j in range(species[s]['breedAmountAllowed']):
            print('making offspring #{}'.format(i+1))
            if (random.random() <= mutateNoCrossoverChance or len(species[s]['list'])==1): # either the genome is a crossover or just the same genome 
                print('Chose no crossover')
                g = Genome.copyGenome(random.choice(species[s]['list']))
            else:
                print('Chose crossover')
                g1 = random.choice(species[s]['list'])
                g2 = random.choice(species[s]['list'])
                g = crossover(g1, g2)
            
            #print('Printing node genes:')
            #g.printNodeGenes()
            #print('Printing connect genes:')
            #g.printConnectGenes()
            
            # add a chance to mutate the connections
            if (g.connectGenes):
                if (random.random() <= genomeMutationChance):
                    print('mutation genome connections !')
                    for connection in g.connectGenes:
                        if (random.random() <= connectionUniformPerturb): # uniformly perturb the connection
                            g.connectGenes[connection][1] += random.uniform(-perturbVal, perturbVal)
                        else:
                            g.connectGenes[connection][1] = random.uniform(-1.0, 1.0) # set a new value for 
            
                
            # chance to add new connections/nodes
            if (random.random() <= largePopulationNewNodeChance):
                print('trying to add new node')
                g.mutateAddRandomNode()
            if (random.random() <= largePopulationNewConnectionChance):
                print('trying to add new connection')
                g.mutateAddRandomConnection()
                
            newPop[i+1] = g
            i+=1
            
    # -----------------------------------------------------------------------------------------------------        
    
    if (leftovers > 0): 
        print('now for the leftovers:')
    for k in range(leftovers): # breeding the leftover genomes that were not calculated through floor
        if (random.random() <= interspeciesMatingRate): # very small chance
            print('interspecies')
            speciesKey1 = random.choice(list(species))
            speciesKey2 = random.choice(list(species))
            g1 = random.choice(species[speciesKey1]['list'])
            g2 = random.choice(species[speciesKey2]['list'])
            g = crossover(g1, g2)
            if (g.connectGenes):
                if (random.random() <= genomeMutationChance):
                    print('mutation genome connections !')
                    for connection in g.connectGenes:
                        if (random.random() <= connectionUniformPerturb): # uniformly perturb the connection
                            g.connectGenes[connection][1] += random.uniform(-perturbVal, perturbVal)
                        else:
                            g.connectGenes[connection][1] = random.uniform(-1.0, 1.0) # set a new value for 
            
                
            # chance to add new connections/nodes
            if (random.random() <= largePopulationNewNodeChance):
                print('trying to add new node')
                g.mutateAddRandomNode()
            if (random.random() <= largePopulationNewConnectionChance):
                print('trying to add new connection')
                g.mutateAddRandomConnection()
                
                
            newPop[i+1] = g
            i+=1
        else: # else just find a random species and mate from there 
            print('random species mating')
            speciesKey = random.choice(list(species))
            if (random.random() <= mutateNoCrossoverChance or len(species[speciesKey]['list']) < 2): # either the genome is a crossover or just the same genome 
                print('Chose no crossover')
                g = Genome.copyGenome(random.choice(species[speciesKey]['list']))
            else:
                print('Chose crossover')
                g1 = random.choice(species[speciesKey]['list'])
                g2 = random.choice(species[speciesKey]['list'])
                g = crossover(g1, g2)
            
            #print('Printing node genes:')
            #g.printNodeGenes()
            #print('Printing connect genes:')
            #g.printConnectGenes()
            
            # add a chance to mutate the connections
            if (g.connectGenes):
                if (random.random() <= genomeMutationChance):
                    print('mutation genome connections !')
                    for connection in g.connectGenes:
                        if (random.random() <= connectionUniformPerturb): # uniformly perturb the connection
                            g.connectGenes[connection][1] += random.uniform(-perturbVal, perturbVal)
                        else:
                            g.connectGenes[connection][1] = random.uniform(-1.0, 1.0) # set a new value for 
            
                
            # chance to add new connections/nodes
            if (random.random() <= largePopulationNewNodeChance):
                print('trying to add new node')
                g.mutateAddRandomNode()
            if (random.random() <= largePopulationNewConnectionChance):
                print('trying to add new connection')
                g.mutateAddRandomConnection()
                
                
            newPop[i+1] = g
            i+=1
            
    
    print('Final value of i: {}'.format(i))
    
    # ------------------------------------------------------------------------------------------
        
    # each existing species is represented by a random genome inside the species from the previous generation 
    if (generation >= 0):
        for s in species:
            species[s] = {'summedSharedFitness': 0.0, 'maxFitness': 0.0, 'maxAdjustedFitness': 0.0, 'stagnantCounter': species[s]['stagnantCounter']+1, 'isStagnant': False, 'list' : [random.choice(species[s]['list'])]}
            species[s]['bestGenome'] = species[s]['list'][0]
            if species[s]['stagnantCounter'] >= numUntilStagnant:  # if the maximum fitness of a species did not improve in 15 generations, the networks in the stagnant species were not allowed to reproduce 
                species[s]['isStagnant'] = True
    
    
    
    
    pop.clear()
    pop = newPop.copy()
    
print("--- %s seconds ---" % (time.time() - start_time))    
print('Total number of evaluations: {}'.format(globalLoopCounter))    
env.close()

Beginning Generation #1
Beginning evaluation on genome #1
creating new species:1
1
Episode finished after 77 timesteps
Genome fitness: -113.16213624092441
Beginning evaluation on genome #2
inserting into species#1
Episode finished after 70 timesteps
Genome fitness: -111.66043783031652
Beginning evaluation on genome #3
inserting into species#1
Episode finished after 148 timesteps
Genome fitness: -105.04635746948912
Beginning evaluation on genome #4
inserting into species#1
Episode finished after 51 timesteps
Genome fitness: -108.45831207266201
Beginning evaluation on genome #5
inserting into species#1
Episode finished after 1601 timesteps
Genome fitness: -47.88970151104202
Beginning evaluation on genome #6
inserting into species#1
Episode finished after 109 timesteps
Genome fitness: -113.93839062311439
Beginning evaluation on genome #7
inserting into species#1
Episode finished after 152 timesteps
Genome fitness: -104.90662131556304
Beginning evaluation on genome #8
inserting into specie

Episode finished after 145 timesteps
Genome fitness: -118.69705538797444
Beginning evaluation on genome #64
inserting into species#1
Episode finished after 54 timesteps
Genome fitness: -110.1827883241487
Beginning evaluation on genome #65
inserting into species#1
Episode finished after 112 timesteps
Genome fitness: -104.75983365788433
Beginning evaluation on genome #66
inserting into species#1
Episode finished after 1601 timesteps
Genome fitness: -45.54719174749398
Beginning evaluation on genome #67
inserting into species#1
Episode finished after 152 timesteps
Genome fitness: -111.36733504288519
Beginning evaluation on genome #68
inserting into species#1
Episode finished after 142 timesteps
Genome fitness: -105.1757007868308
Beginning evaluation on genome #69
inserting into species#1
Episode finished after 128 timesteps
Genome fitness: -116.48422264912041
Beginning evaluation on genome #70
inserting into species#1
Episode finished after 143 timesteps
Genome fitness: -105.48050274977952

Episode finished after 171 timesteps
Genome fitness: -103.17743590629468
Beginning evaluation on genome #126
inserting into species#1
Episode finished after 73 timesteps
Genome fitness: -96.33932100929816
Beginning evaluation on genome #127
inserting into species#1
Episode finished after 134 timesteps
Genome fitness: -107.51566520496647
Beginning evaluation on genome #128
inserting into species#1
Episode finished after 1601 timesteps
Genome fitness: -50.12646389468424
Beginning evaluation on genome #129
inserting into species#1
Episode finished after 144 timesteps
Genome fitness: -105.41718888676162
Beginning evaluation on genome #130
inserting into species#1
Episode finished after 1601 timesteps
Genome fitness: -45.794043820303486
Beginning evaluation on genome #131
inserting into species#1
Episode finished after 94 timesteps
Genome fitness: -105.92089163810695
Beginning evaluation on genome #132
inserting into species#1
Episode finished after 1601 timesteps
Genome fitness: -47.747806

Episode finished after 108 timesteps
Genome fitness: -116.6542827644994
Beginning evaluation on genome #188
inserting into species#1
Episode finished after 97 timesteps
Genome fitness: -105.45963280439064
Beginning evaluation on genome #189
inserting into species#1
Episode finished after 110 timesteps
Genome fitness: -115.96730266599234
Beginning evaluation on genome #190
inserting into species#1
Episode finished after 116 timesteps
Genome fitness: -105.46692817405918
Beginning evaluation on genome #191
inserting into species#1
Episode finished after 66 timesteps
Genome fitness: -112.33212982042818
Beginning evaluation on genome #192
inserting into species#1
Episode finished after 64 timesteps
Genome fitness: -113.53092692827869
Beginning evaluation on genome #193
inserting into species#1
Episode finished after 82 timesteps
Genome fitness: -106.00586329929219
Beginning evaluation on genome #194
inserting into species#1
Episode finished after 107 timesteps
Genome fitness: -104.368709732

Episode finished after 119 timesteps
Genome fitness: -104.77979607143499
Beginning evaluation on genome #250
inserting into species#1
Episode finished after 139 timesteps
Genome fitness: -105.78256593119721
Beginning evaluation on genome #251
inserting into species#1
Episode finished after 102 timesteps
Genome fitness: -104.24086404656988
Beginning evaluation on genome #252
inserting into species#1
Episode finished after 150 timesteps
Genome fitness: -102.48354362206469
Beginning evaluation on genome #253
inserting into species#1
Episode finished after 102 timesteps
Genome fitness: -113.88829305988426
Beginning evaluation on genome #254
inserting into species#1
Episode finished after 48 timesteps
Genome fitness: -109.16264064289815
Beginning evaluation on genome #255
inserting into species#1
Episode finished after 121 timesteps
Genome fitness: -104.40390941739824
Beginning evaluation on genome #256
inserting into species#1
Episode finished after 130 timesteps
Genome fitness: -114.65778

...

Episode finished after 98 timesteps
Genome fitness: -117.89260019120948
Beginning evaluation on genome #250
inserting into species#1
Episode finished after 98 timesteps
Genome fitness: -116.56714777027135
Beginning evaluation on genome #251
inserting into species#1
Episode finished after 148 timesteps
Genome fitness: -117.62772598219601
Beginning evaluation on genome #252
inserting into species#1
Episode finished after 76 timesteps
Genome fitness: -116.38362014921817
Beginning evaluation on genome #253
inserting into species#1
Episode finished after 160 timesteps
Genome fitness: -117.70272946773036
Beginning evaluation on genome #254
inserting into species#1
Episode finished after 156 timesteps
Genome fitness: -105.75858532304063
Beginning evaluation on genome #255
inserting into species#1
Episode finished after 117 timesteps
Genome fitness: -118.2481088354532
Beginning evaluation on genome #256
inserting into species#1
Episode finished after 102 timesteps
Genome fitness: -117.54428968

added hidden node
making offspring #288
Chose crossover
trying to add new connection
Attempted to connect too many times. probably already full
making offspring #289
Chose no crossover
trying to add new connection
Attempted to connect too many times. probably already full
making offspring #290
Chose crossover
mutation genome connections !
making offspring #291
Chose crossover
mutation genome connections !
trying to add new connection
Attempted to connect too many times. probably already full
making offspring #292
Chose no crossover
mutation genome connections !
making offspring #293
Chose crossover
making offspring #294
Chose no crossover
mutation genome connections !
making offspring #295
Chose no crossover
making offspring #296
Chose no crossover
making offspring #297
Chose crossover
making offspring #298
Chose crossover
mutation genome connections !
making offspring #299
Chose crossover
making offspring #300
Chose crossover
Final value of i: 300
Beginning Generation #10
Beginning ev

Episode finished after 147 timesteps
Genome fitness: -117.75978748766396
Beginning evaluation on genome #56
inserting into species#1
Episode finished after 105 timesteps
Genome fitness: -104.55434790874455
Beginning evaluation on genome #57
inserting into species#1
Episode finished after 153 timesteps
Genome fitness: -117.86405301656451
Beginning evaluation on genome #58
inserting into species#1
Episode finished after 74 timesteps
Genome fitness: -113.82849709047068
Beginning evaluation on genome #59
inserting into species#1
Episode finished after 121 timesteps
Genome fitness: -116.16130685991111
Beginning evaluation on genome #60
inserting into species#1
Episode finished after 96 timesteps
Genome fitness: -112.01490566260307
Beginning evaluation on genome #61
inserting into species#1
Episode finished after 48 timesteps
Genome fitness: -109.62178355499978
Beginning evaluation on genome #62
inserting into species#1
Episode finished after 102 timesteps
Genome fitness: -117.65546484488685

Episode finished after 87 timesteps
Genome fitness: -116.40612585558867
Beginning evaluation on genome #118
inserting into species#1
Episode finished after 120 timesteps
Genome fitness: -117.16387202512722
Beginning evaluation on genome #119
inserting into species#1
Episode finished after 100 timesteps
Genome fitness: -116.50633576222634
Beginning evaluation on genome #120
inserting into species#1
Episode finished after 94 timesteps
Genome fitness: -116.7322627060339
Beginning evaluation on genome #121
inserting into species#1
Episode finished after 122 timesteps
Genome fitness: -117.18695141939136
Beginning evaluation on genome #122
inserting into species#1
Episode finished after 103 timesteps
Genome fitness: -117.59625732233314
Beginning evaluation on genome #123
inserting into species#1
Episode finished after 1601 timesteps
Genome fitness: -47.25721229002673
Beginning evaluation on genome #124
inserting into species#1
Episode finished after 159 timesteps
Genome fitness: -117.8758697

Episode finished after 74 timesteps
Genome fitness: -113.141274069064
Beginning evaluation on genome #180
inserting into species#1
Episode finished after 81 timesteps
Genome fitness: -106.44707629668648
Beginning evaluation on genome #181
inserting into species#1
Episode finished after 102 timesteps
Genome fitness: -117.21659085491363
Beginning evaluation on genome #182
inserting into species#1
Episode finished after 97 timesteps
Genome fitness: -118.11229162547428
Beginning evaluation on genome #183
inserting into species#1
Episode finished after 117 timesteps
Genome fitness: -116.83027006913599
Beginning evaluation on genome #184
inserting into species#1
Episode finished after 104 timesteps
Genome fitness: -116.39415967356787
Beginning evaluation on genome #185
inserting into species#1
Episode finished after 59 timesteps
Genome fitness: -110.93976982118315
Beginning evaluation on genome #186
inserting into species#1
Episode finished after 63 timesteps
Genome fitness: -112.08620932952

In [ ]:
print(bestGenome.nodeGenes)

In [ ]:
# test bench 

env = gym.make(envName)

testBenchTotalFitness = 0

for i in range(100):
    t = 0
    bestGenome.resetFitness()
    observation = env.reset()
    while(True):
        t+=1
        env.render()
        action = bestGenome.evaluateAction(observation)
        observation, reward, done, info = env.step(action)
        bestGenome.fitness += reward
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            testBenchTotalFitness+=bestGenome.fitness
            print("Genome fitness: {}".format(bestGenome.fitness))
            break

print('average fitness across 100 trials:{}'.format(testBenchTotalFitness/100))
env.close()